In [1]:
from typing import *
import json
import os

from easydict import EasyDict as edict
import pymysql as sql

In [2]:
SEP       = os.path.sep
ROOT_PATH = SEP.join(os.getcwd().split(SEP)[:-5])
JSON_PATH = f'{ROOT_PATH}/utils/configs'

CONFIGS   = json.loads(open(f'{JSON_PATH}/config.json', 'r').read())
PORTS     = json.loads(open(f'{JSON_PATH}/ports.json', 'r').read())

CONFIGS   = edict(CONFIGS)
PORTS     = edict(PORTS)

In [3]:
host      = CONFIGS.global_host
port      = PORTS.sql_port
user      = CONFIGS.sql_user
passwd    = CONFIGS.sql_passwd

In [4]:
conn      = sql.connect(host = host, password = passwd, user = user,
                        port = port, db       = 'DoveNest')
cursor    = conn.cursor()

In [5]:
def print_result(results: tuple):

    for idx, result in enumerate(results, 1): print(f'[{str(idx).zfill(3)}] {result}')
    print('\n')


def select(cursor, table_name: str, column: str = '*', 
           order  : Union[None, str] = None, cond:  Union[None, str] = None,
           limit_k: Union[None, str] = None, group: Union[None, str] = None):

    query = f'select distinct {column} from {table_name}'

    if    cond: query += f' where {cond}'
    if   group: query += f' group by {group}'
    if   order: query += f' order by {order}'
    if limit_k: query += f' limit {limit_k}'

    print(f'[QUERY] {query};')
    cursor.execute(query)

    print_result(cursor.fetchall())

In [9]:
select(cursor, 'discount_info', column = 'original, discounted, percent, date',
       cond  = '(platform = "nintendo") and (date like "202308%")', limit_k = '15',
       order = 'percent desc, title asc, idx asc')

[QUERY] select distinct original, discounted, percent, date from discount_info where (platform = "nintendo") and (date like "202308%") order by percent desc, title asc, idx asc limit 15;
[001] ('₩ 39,000', '₩ 1,000', 97, '20230801')
[002] ('₩ 33,990', '₩ 1,010', 97, '20230801')
[003] ('₩ 25,000', '₩ 1,000', 96, '20230801')
[004] ('₩ 19,990', '₩ 1,000', 95, '20230801')
[005] ('₩ 19,990', '₩ 1,030', 95, '20230801')
[006] ('₩ 20,500', '₩ 1,000', 95, '20230801')
[007] ('₩ 18,990', '₩ 1,000', 95, '20230801')
[008] ('₩ 25,000', '₩ 1,250', 95, '20230801')
[009] ('₩ 16,200', '₩ 1,000', 94, '20230801')
[010] ('₩ 15,990', '₩ 1,000', 94, '20230801')
[011] ('₩ 16,700', '₩ 1,000', 94, '20230801')
[012] ('₩ 33,430', '₩ 1,990', 94, '20230801')
[013] ('₩ 14,990', '₩ 1,000', 93, '20230801')
[014] ('₩ 13,990', '₩ 1,000', 93, '20230801')
[015] ('₩ 15,000', '₩ 1,050', 93, '20230801')




In [7]:
select(cursor, 'discount_info', column = 'platform', group = 'platform')

[QUERY] select distinct platform from discount_info group by platform;
[001] ('nintendo',)
[002] ('steam',)




In [8]:
cursor.execute('show columns from discount_info')
cursor.fetchall()

(('idx', 'int(11)', 'NO', '', None, ''),
 ('appid', 'text', 'NO', '', None, ''),
 ('title', 'text', 'NO', '', None, ''),
 ('percent', 'int(11)', 'NO', '', None, ''),
 ('discounted', 'text', 'NO', '', None, ''),
 ('original', 'text', 'NO', '', None, ''),
 ('platform', 'text', 'NO', '', None, ''),
 ('storepage', 'text', 'YES', '', None, ''),
 ('thumbnail', 'text', 'YES', '', None, ''),
 ('date', 'text', 'NO', '', None, ''),
 ('review', 'text', 'YES', '', None, ''))